In [1]:
import pandas as pd
import py_stringmatching as sm
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
from sklearn.cross_validation import train_test_split
import re
import operator
style.use('ggplot')

# read in csv as dataframe
# c = pd.read_csv("../blocking/candidate_set.csv", encoding="ISO-8859-1", index_col='_id')
c = pd.read_csv("../blocking/candidate_set_with_added_features.csv", encoding="ISO-8859-1", index_col='_id')

c.head()

/Users/andrew/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,ltable_id,rtable_id,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,ltable_conservation_paragraph,ltable_pop_trend,...,countries_countries_lev_sim,countries_countries_nmw,countries_countries_sw,country_count_country_count_exm,country_count_country_count_anm,country_count_country_count_lev_dist,country_count_country_count_lev_sim,status_match,country_overlap,country_count_sim
_id,,,,,,,,,,,,,,,,,,,,,
25215,1470,3,"adelaide pigmy bluetongue skink, pygmy bluetongue",Tiliqua,Scincidae,NaN,Australia;,NaN,NaN,NaN,...,0.16,-10.0,3.0,0,0.5,1.0,0.0,1,0.0,1
75598,4363,9,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\r\r\r\n The major threat is drainage of we...,"[""\n It is listed on Appendix III of the Be...",Decreasing,...,0.16,-2.0,10.0,1,1.0,0.0,1.0,1,1.0,1
75600,4365,9,NaN,Pelophylax,Ranidae,Terrestrial; Freshwater,China;,\r\r\r\n The major threat is over-collectin...,['\n The Shapotou location is within a nati...,Decreasing,...,0.08,-16.0,1.0,0,0.5,1.0,0.0,1,0.0,1
78778,7543,9,karpathos frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Greece;,\r\r\r\n The major threat is habitat loss a...,['\n It is listed on Appendix III of the Be...,Decreasing,...,0.12,-14.0,2.0,0,0.5,1.0,0.0,0,0.0,1
83513,4363,10,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\r\r\r\n The major threat is drainage of we...,"[""\n It is listed on Appendix III of the Be...",Decreasing,...,0.16,-2.0,10.0,1,1.0,0.0,1.0,1,1.0,1


In [2]:
c.columns

Index(['ltable_id', 'rtable_id', 'ltable_name', 'ltable_genus',
       'ltable_family', 'ltable_ecology', 'ltable_countries',
       'ltable_threat_paragraph', 'ltable_conservation_paragraph',
       'ltable_pop_trend', 'ltable_status', 'ltable_country_count',
       'ltable_scientific_name', 'rtable_scientific_name', 'rtable_name',
       'rtable_kingdom', 'rtable_phylum', 'rtable_class', 'rtable_order',
       'rtable_family', 'rtable_genus', 'rtable_size', 'rtable_threats',
       'rtable_conservation', 'rtable_threat_keywords',
       'rtable_conservation_keywords', 'rtable_status', 'rtable_countries',
       'rtable_country_count', 'rtable_tCount', '_id.1', 'ltable_id.1',
       'rtable_id.1', 'name_name_jac_qgm_3_qgm_3',
       'name_name_jac_dlm_dc0_dlm_dc0', 'countries_countries_jac_qgm_3_qgm_3',
       'countries_countries_cos_dlm_dc0_dlm_dc0',
       'countries_countries_jac_dlm_dc0_dlm_dc0', 'countries_countries_mel',
       'countries_countries_lev_dist', 'countries_countri

In [13]:
def list_duplicates(seq):
  seen = set()
  seen_add = seen.add
  # adds all elements it doesn't know yet to seen and all other to seen_twice
  seen_twice = set( x for x in seq if x in seen or seen_add(x) )
  # turn the set into a list (as requested)
  return list( seen_twice )

list_duplicates(c.columns)

[]

In [3]:
matches = 0
nonmatches = 0;

gold = None
DS = None

while matches < 50:
    print('matches:', matches)
    gold,unlabeled = train_test_split(c, train_size=0.1551, test_size=0.8449)

    #Automatically label the training data
    labels = []
    matches = 0;
    nonmatches = 0;
    
    for index, row in gold.iterrows():

        if row['rtable_scientific_name'].strip().lower() == row['ltable_scientific_name'].strip().lower():
            labels.append(1)
            matches = matches + 1
        else:
            labels.append(0)
            nonmatches = nonmatches + 1

#     matches = 50
gold['label'] = labels
print('number of matches in training set:', matches)
print('number of nonmatches in training set:', nonmatches)

matches: 0
matches: 35
matches: 31
matches: 39
matches: 49
matches: 40
matches: 41
matches: 38
matches: 39
matches: 38
matches: 42
matches: 44
matches: 45
matches: 40
matches: 43
matches: 37
matches: 37
matches: 41
matches: 48
matches: 39
matches: 44
matches: 43
matches: 44
matches: 41
matches: 35
matches: 45
matches: 38
matches: 45
matches: 42
matches: 38
matches: 25
number of matches in training set: 52
number of nonmatches in training set: 347


/Users/andrew/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
for col in c.columns:
    print(col)

ltable_id
rtable_id
ltable_name
ltable_genus
ltable_family
ltable_ecology
ltable_countries
ltable_threat_paragraph
ltable_conservation_paragraph
ltable_pop_trend
ltable_status
ltable_country_count
ltable_scientific_name
rtable_scientific_name
rtable_name
rtable_kingdom
rtable_phylum
rtable_class
rtable_order
rtable_family
rtable_genus
rtable_size
rtable_threats
rtable_conservation
rtable_threat_keywords
rtable_conservation_keywords
rtable_status
rtable_countries
rtable_country_count
rtable_tCount
_id.1
ltable_id.1
rtable_id.1
name_name_jac_qgm_3_qgm_3
name_name_jac_dlm_dc0_dlm_dc0
countries_countries_jac_qgm_3_qgm_3
countries_countries_cos_dlm_dc0_dlm_dc0
countries_countries_jac_dlm_dc0_dlm_dc0
countries_countries_mel
countries_countries_lev_dist
countries_countries_lev_sim
countries_countries_nmw
countries_countries_sw
country_count_country_count_exm
country_count_country_count_anm
country_count_country_count_lev_dist
country_count_country_count_lev_sim
status_match
country_overlap
co

In [5]:
# gold.to_csv('./gold.csv')
gold.to_csv('./gold.csv')
# unlabeled.to_csv('./unlabeled.csv')
unlabeled.to_csv('./unlabeled.csv')


In [14]:
# remove unnecessary features
# write out gold and unlabeled csvs with only selected features for matcher

cols_to_keep = ['country_count_sim', 
                'status_match', 
                'country_overlap', 
                'name_name_jac_dlm_dc0_dlm_dc0', 
                'name_name_jac_qgm_3_qgm_3',
                'countries_countries_jac_dlm_dc0_dlm_dc0', 
                'countries_countries_jac_qgm_3_qgm_3'
                ]

# FOR NOW, just use these while we wait for other features
# cols_to_keep_gold = ['country_count_sim', 
#                 'name_name_jac_dlm_dc0_dlm_dc0', 
#                 'name_name_jac_qgm_3_qgm_3',
#                 'label']
gold_reduced = gold[cols_to_keep + ['label']] 

# cols_to_keep_unlabeled = ['country_count_sim', 
#                 'name_name_jac_dlm_dc0_dlm_dc0', 
#                 'name_name_jac_qgm_3_qgm_3']
unlabeled_reduced = unlabeled[cols_to_keep]

In [15]:
# gold_reduced.to_csv('./gold_reduced.csv')
gold_reduced.to_csv('./gold_reduced.csv')
# unlabeled_reduced.to_csv('./unlabeled_reduced.csv')
unlabeled_reduced.to_csv('./unlabeled_reduced.csv')

In [16]:
#using features for matcher
train,test = train_test_split(gold, train_size=0.7, test_size=0.3)
len(train)

train_reduced = train[cols_to_keep] 
test_reduced = test[cols_to_keep]

In [17]:
train.to_csv('./train.csv')
test.to_csv('./test.csv')
train_reduced.to_csv('./train_reduced.csv')
test_reduced.to_csv('./test_reduced.csv')

In [19]:
train_reduced

,country_count_sim,status_match,country_overlap,name_name_jac_dlm_dc0_dlm_dc0,name_name_jac_qgm_3_qgm_3,countries_countries_jac_dlm_dc0_dlm_dc0,countries_countries_jac_qgm_3_qgm_3
_id,,,,,,,
5766433,1,0,0.0,NaN,NaN,0.0,0.000000
2103733,1,0,0.0,NaN,NaN,0.0,0.000000
8527301,0,0,0.0,0.250000,0.222222,0.0,0.000000
6621928,1,1,0.0,NaN,NaN,0.0,0.000000
8998313,1,1,0.0,0.500000,0.150000,0.0,0.000000
847440,0,1,0.0,0.250000,0.121212,0.0,0.018519
8862181,1,1,0.0,0.500000,0.575000,0.0,0.000000
2103595,0,0,0.0,NaN,NaN,0.0,0.000000
8235396,1,0,0.0,NaN,NaN,0.0,0.000000
